In [94]:
import numpy as np
from figaro.utils import rejection_sampler
from figaro.cosmology import Planck18 as Omega, dVdz_approx_planck18
import matplotlib.pyplot as plt

In [95]:
true_H0 = 70. # km/s/Mpc

In [96]:
# Mass distribution
from population_models.mass import plpeak

In [97]:
# Generate true detector frame mass samples
def generate_true_mz(n_SE_draws):
    M  = rejection_sampler(n_SE_draws, plpeak, [0,200])
    z  = rejection_sampler(n_SE_draws, lambda z: dVdz_approx_planck18(z)*np.exp(-z/0.1)/(1+z), [0, 0.5])
    # z  = rejection_sampler(n_SE_draws, lambda z: dVdz_approx_planck18(z)/(1+z), [0, 2])
    dL = Omega.LuminosityDistance(z)
    return np.array([M * (1 + z), dL]).T

In [98]:
# Generate detector frame mass posterior samples
def generate_posterior_samples(truth, sigma = 0.1, n_samples = 1000):
    log_obs_mean = np.log(truth) + np.random.normal(0, sigma, truth.shape) # shape = (n_events, n_params)
    return np.exp(np.random.normal(log_obs_mean, sigma, (n_samples, truth.shape[0], truth.shape[1]))).transpose(1,2,0) # shape = (n_events, n_params, n_samples)

In [99]:
from selection_function import selection_function

n_SE_draws = 1000
outdir = 'hierarchical_SE_test_3'

samples = generate_true_mz(n_SE_draws)
posterior = generate_posterior_samples(samples[np.random.uniform(0,1,samples.shape[0]) <= selection_function(samples)])

samples.shape, posterior.shape

((1000, 2), (245, 2, 1000))

In [100]:
np.savetxt(outdir+'/true_samples.txt', samples)

In [101]:
posterior.max(axis=(0,2))

array([ 145.42364148, 3897.67898786])

In [102]:
np.savetxt(outdir+'/jsd_bounds.txt', np.array([np.median(posterior, axis=2).min(axis=0), np.median(posterior, axis=2).max(axis=0)]).transpose())

In [103]:
import os
if not os.path.exists(outdir+'/data'):
    os.makedirs(outdir+'/data')
for i,p in enumerate(posterior):
    np.savetxt(f'{outdir}/data/posterior_samples_{i+1}.txt', p.T)